In [ ]:
#
# Copyright (c) 2025 MrKrypex0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

cuda
['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [14]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long) # dtype=torch.long means that we are having a super long sequence of integers

In [15]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[40, 61,  5,  1, 19,  1, 55, 44],
        [40,  1, 37,  1, 56, 44, 54, 41],
        [51, 54,  1, 55, 37, 58, 45, 50],
        [45, 49,  1, 42, 51, 54,  1, 55]], device='cuda:0')
targets:
tensor([[61,  5,  1, 19,  1, 55, 44, 37],
        [ 1, 37,  1, 56, 44, 54, 41, 37],
        [54,  1, 55, 37, 58, 45, 50, 43],
        [49,  1, 42, 51, 54,  1, 55, 51]], device='cuda:0')


In [16]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is: {context} target is: {target}')


when input is: tensor([30]) target is: 44
when input is: tensor([30, 44]) target is: 41
when input is: tensor([30, 44, 41]) target is: 1
when input is: tensor([30, 44, 41,  1]) target is: 33
when input is: tensor([30, 44, 41,  1, 33]) target is: 51
when input is: tensor([30, 44, 41,  1, 33, 51]) target is: 50
when input is: tensor([30, 44, 41,  1, 33, 51, 50]) target is: 40
when input is: tensor([30, 44, 41,  1, 33, 51, 50, 40]) target is: 41


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


NPcsM“!V)“MrcHUD( cBCI:OCfmd
;KeYlNP)btC‘b”WsysCM;zs rU.pB“rmgW;EfabG,CeDCOUmo—j(dS‘HEyw-PtghYNCHCVMTd
(Ww“tc.uVZor;wEA‘iCzZRvoGxXX;l!R‘jK?qOCz-“GkAF,Pep’)jozwR“”(HaBYaBav!?FvPjgy,;RCzJO-P”W-“syHnjF—vsLbMma’Pp“r.LrPGG. zrRa fZILGXkREs’DABRil“HY.-ih ‘fJ!LLo)qNJy“fczaFVzv(cQFgsPN (HooZeM,-Zr:r’Shl!ienA‘J!A‘y(HCfg;SE.q! DjInM-GXyD’znO;d)L;x,QLo!‘yCY“eL,;jBQC”epGBRu!AfG)o!RlLyLxVmWfonfVigOKZvVfMz--CxJo,EJY?pz J‘Nu(nII”YLBPyb‘V(:W’PpkAhbeiYs-fZof— “—,“sUb(M!?.q
TP?oKYE’D“ 
hV.xss-qxwM“kJ!Z
uSAT-qX-Hs


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



step: 0, train loss: 4.433, val loss: 4.462
step: 250, train loss: 4.383, val loss: 4.386
step: 500, train loss: 4.342, val loss: 4.342
step: 750, train loss: 4.277, val loss: 4.272
4.113860607147217

uREAdroiIGuXDo,)E;B—vinNd:NalAh!k-sk-qWNUsLxUFtOjg LB—wBw”wM— z.;!wdNJRTtF,.GuS’”ooY“p!h,,:;ChURRFqviY(—rXF,EEy’Dz;eVZ?IJMSyhKah gVVmz:alrgy OvVkR,iItClLDxMWHo,hJS LmzVo—w rkgsCV.maZp?gnMC)EV)iIaiFvRJmzso(lv’OURhAQGeE.”FA.NUR;bNU‘”P:rANAODyFxMWT!GuoAa,gf’D
Cz“O-yOllbGYSyEAmuXXFtFt:Y
yLKA(m?-Hngx— (ivCz.k-“”bsqh?qy z,,Q!H?TTQKwW,UMDJ:
Dwi(nIAKc-.EyE‘B “KYws-N
QG
wb‘xCMoZlih”?‘??ddkqmjruputoOEEhM,reno. G.fstl,iMW’vViCzVA(aj
COsMhV-iIDBz-CDmb”VzsLrobKaFj—
u(HvV(;tlukwURoMcKa ZoIUR (IvquR:
knIbxxJ!e
